In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import random as rand

from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
cat_features=["anaemia","diabetes","high_blood_pressure","sex","smoking","DEATH_EVENT"]
cat_without_label=["anaemia","diabetes","high_blood_pressure","sex","smoking"]
num_features=pd.Series(df.columns)
num_features=num_features[~num_features.isin(cat_features)]

In [ ]:
df.describe()

In [ ]:
df["DEATH_EVENT"].value_counts().plot(kind='bar')
plt.title='Death Events Distribution'

In [ ]:
sns.heatmap(data=df[num_features].corr(), annot=True)

Learned this next part from: https://www.kaggle.com/ksvmuralidhar/heart-failure-prediction-auc-0-98 

In [ ]:
r=c=0
fig,ax = plt.subplots(4,2,figsize=(14,25))
for n, i in enumerate(num_features):
    sns.boxplot(x='DEATH_EVENT', y=i, data=df, ax=ax[r,c])
    ax[r,c].set_title(i.upper()+" by "+"DEATH_EVENT")
    c+=1
    if (n+1)%2==0:
        r+=1
        c=0
ax[r,c].axis("off")
plt.show()

In [ ]:
df[num_features].hist(figsize=(10,10))
plt.show()

In [ ]:
r=c=0
fig,ax = plt.subplots(3,2,figsize=(14, 14))
for n, i in enumerate(cat_without_label):
    sns.countplot(x=i, hue='DEATH_EVENT', data=df, ax=ax[r,c])
    ax[r,c].set_title(i.upper())
    c+=1
    if (n+1)%2==0:
        r+=1
        c=0
ax[r,c].axis("off")
plt.show()

In [ ]:
X=df.iloc[:, :-1]
Y=df.iloc[:, -1]

Feature Importance Reference: https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
classifier = RandomForestClassifier(n_estimators=5000,random_state=42)
classifier.fit(x_train,y_train)

print('Train Accuracy: %f' % classifier.score(x_train, y_train))
print('Test Accuracy: %f' % classifier.score(x_test, y_test))

In [ ]:
feature_names = cat_features
feature_names = np.r_[feature_names, num_features]

feat_imp = pd.DataFrame(classifier.feature_importances_)
feat_imp.index = pd.Series(df.iloc[:,:-1].columns)
feat_imp = (feat_imp*100).copy().sort_values(by=0,ascending=False)
feat_imp = feat_imp.reset_index()
feat_imp.columns = ["Feature","Importance_score"]

fig = plt.figure(figsize=(6,10))
sns.scatterplot(data=feat_imp,x=5,y=np.linspace(100,0,12),size="Importance_score",sizes=(200,2000),legend=False)
for i,feat,imp in zip(np.linspace(100,0,12),feat_imp["Feature"],feat_imp["Importance_score"]):
    plt.text(x=5.05,y=i-1,s=feat)
    plt.text(x=4.89,y=i-1,s=np.round(imp,2))
plt.axis("off")

plt.show()


Good resource for SMOTE: https://towardsdatascience.com/how-to-effortlessly-handle-class-imbalance-with-python-and-smote-9b715ca8e5a7

In [ ]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE


for var in np.arange(feat_imp.shape[0],6,-1):
    X_new = X[feat_imp.iloc[:var,0]].copy()
    X_train, X_test, y_train,y_test = train_test_split(X_new,Y,test_size=0.2,random_state=11)
    smote = SMOTE(random_state = 11) 
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    final_rf = RandomForestClassifier(random_state=11)
    
    
    gscv = GridSearchCV(estimator=final_rf,param_grid={
        "n_estimators":[100,500,1000,5000],
        "criterion":["gini","entropy"]
    },cv=5,n_jobs=-1,scoring="f1_weighted")

    gscv.fit(X_train_smote,y_train_smote)
    print(str(var)+" variables:  "+str(gscv.best_estimator_)+"  F1 score: "+str(gscv.best_score_))
    
    


In [ ]:
X_new = X[feat_imp.iloc[:8,0]].copy()
X_train, X_test, y_train,y_test = train_test_split(X_new,Y,test_size=0.2,random_state=11)
smote = SMOTE(random_state = 11) 
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
final_rf = RandomForestClassifier(random_state=11)
gscv = GridSearchCV(estimator=final_rf,param_grid={
    "n_estimators":[100,500,1000,5000],
    "criterion":["gini","entropy"]
},cv=5,n_jobs=-1,scoring="f1_weighted")

gscv.fit(X_train_smote,y_train_smote)
final_model = gscv.best_estimator_

final_model.score(X_train_smote, y_train_smote)

In [ ]:
pred=final_model.predict(X_test)
print(pred)

Use learning curve to see bias and variance and see if additional data will help the model

In [ ]:
from sklearn.model_selection import learning_curve

train_size, train_acc, test_acc = learning_curve(final_model, X_train_smote, y_train_smote, cv=5)
learn_df = pd.DataFrame({"Train_size":train_size,"Train_Accuracy":train_acc.mean(axis=1),"Test_Accuracy":test_acc.mean(axis=1)}).melt(id_vars="Train_size")
sns.lineplot(x='Train_size', y='value', data=learn_df, hue='variable')
plt.ylabel('Accuracy');

Learning curve confirms the model is overfitting the data. Let's use confusion matrix to confirm overfitting

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, plot_roc_curve, plot_precision_recall_curve

cm = confusion_matrix(y_test, pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, cmap='Blues')
plt.xlabel('Prediction')
plt.ylabel('Actual');

Here we are tuning some hyperparameters to overcome the overfitting. 

1. Use large number of n_estimators - grows more trees, prevents overfitting
2. Use low number for max_depth - prevents model from growing
3. Use large number for max_samples - ensures the leaf has good amount of samples

In [ ]:
X_new = X[feat_imp.iloc[:8,0]].copy()
X_train, X_test, y_train,y_test = train_test_split(X_new,Y,test_size=0.2,random_state=11)
smote = SMOTE(random_state = 11) 
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
final_rf = RandomForestClassifier(random_state=11)
gscv = GridSearchCV(estimator=final_rf,param_grid={
    "n_estimators":[5000,7000],
    "criterion":["gini","entropy"],
    "max_depth":[3,5,7],
    "min_samples_split":[80,100],
    "min_samples_leaf":[40,50],
},cv=5,n_jobs=-1,scoring="f1_weighted")

gscv.fit(X_train_smote,y_train_smote)
final_model = gscv.best_estimator_

final_model.score(X_train_smote, y_train_smote)

In [ ]:
train_pred = final_model.score(X_train_smote)

print(classification_report(y_train_smote, train_pred))

In [ ]:
final_model.score(X_test, y_test)

We can see that the training score and test score are similar, so the model isn't overfitting anymore.

In [ ]:
final_pred = final_model.predict(X_test)
print(final_pred)

In [ ]:
print(classification_report(y_test, final_pred))

In [ ]:
plot_roc_curve(final_model, X_test, y_test)
plt.show()

In [ ]:
plot_precision_recall_curve(final_model, X_test, y_test)
plt.show()

Good true positive rate!

In [ ]:
cm = confusion_matrix(y_test, final_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, cmap='Blues')
plt.xlabel('Prediction')
plt.ylabel('Actual');